# ***1.Library & Data***

In [ ]:
import pandas as pd 
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor 

# ***2.Exploratory Data Analysis (EDA)***

In [ ]:
train = pd.read_csv('/content/drive/My Drive/Dacon/Data/Prediction number of borrowed Bicycle /train.csv')
test = pd.read_csv('/content/drive/My Drive/Dacon/Data/Prediction number of borrowed Bicycle /test.csv')
submission = pd.read_csv('/content/drive/My Drive/Dacon/Data/Prediction number of borrowed Bicycle /submission.csv')

In [ ]:
train.head()

In [ ]:
test.head() #feature count가 포합되어있지 않음

In [ ]:
submission.head()

+ id : 날짜와 시간별 id
+ hour_bef_temperature : 1시간 전 기온
+ hour_bef_precipitation : 1시간 전 비 정보, 비가 오지 않았으면 0, 비가 오면 1
+ hour_bef_windspeed : 1시간 전 풍속(평균)
+ hour_bef_humidity : 1시간 전 습도
+ hour_bef_visibility : 1시간 전 시정(視程), 시계(視界)(특정 기상 상태에 따른 가시성을 의미)
+ hour_bef_ozone : 1시간 전 오존
+ hour_bef_pm10 : 1시간 전 미세먼지(머리카락 굵기의 1/5에서 1/7 크기의 미세먼지)
+ hour_bef_pm2.5 : 1시간 전 미세먼지(머리카락 굵기의 1/20에서 1/30 크기의 미세먼지)
+ count : 시간에 따른 따릉이 대여 수 

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
train.info() # 결측치 확인 (1459개에서 각 각 모자란 만큼 데이터가 비어있음)

In [ ]:
test.info()

In [ ]:
train.describe() #기술 통계량 확인

In [ ]:
train.groupby('hour').mean()['count'].plot() #시간별 평균 따릉이 대여수 시각화

# 출근시간과 퇴근시간에 사용량이 급격히 증가되는것을 볼수있음

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train.groupby('hour').mean()['count'], 'o-')
plt.grid()
plt.title('count by hour',fontsize=15)
plt.xlabel('hour', fontsize=15)
plt.ylabel('count', fontsize=15)

plt.axvline(8, color='red') # 축을 가로지르는 세로선 생성
plt.axvline(18, color='red')

plt.text(8,120 ,'go to work') #그래프에 텍스트 생성
plt.text(18,120 ,'leave work')

### plt.plot()의 스타일

색깔

|문자열|약자|
|----|-----|
|blue|b|
|green|g|
|red|r|
|cyan|c|
|magenta|m|
|yellow|y|
|black|k|
|white|w|

마커

|마커|의미|
|----|----|
|.|점|
|o|원|
|v|역삼각형|
|^|삼각형|
|s|사각형|
|*|별|
|x|엑스|
|d|다이아몬드|

선

|문자열|의미|
|-----|-----|
| - | 실선|
|-- | 끊어진 실선|
| -.| 점+실선|
|:|점선|

- 상관계수: 두 개의 변수가 같이 일어나는 강도를 나타내는 수치 
- -1에서 1사이의 값을 지닙니다. 
- -1이나 1인 수치는 현실 세계에서 관측되기 힘든 수치입니다. 
- 분야별로 기준을 정하는 것에 따라 달라지겠지만, 보통 0.4이상이면 두 개의 변수간에 상관성이 있다고 얘기합니다. 

![상관계수](https://t1.daumcdn.net/cfile/tistory/99DEE1425C6A9F2008)

###상관계수의 의미
- 상관관계는 인과관계와 다름

![상관성 예시](https://miro.medium.com/max/684/1*JLYI5eCVEN7ZUWXBIrrapw.png)

- 선글라스 판매량이 증가함에 따라, 아이스크림 판매액도 같이 증가. 
- 하지만 선글라스 판매량이 증가했기 **때문에** 아이스크림 판매액이 증가했다라고 해석하는 것은 타당하지 않음. 
- 선글라스 판매량이 증가했다는 것은 여름 때문이라고 볼 수 있으므로, 날씨가 더워짐에 따라 선글라스 판매량과 아이스크림 판매액이 같이 증가했다고 보는 것이 타당. 

* 따라서 이 두 특성간에는 상관관계는 있으나 인과관계로는 볼수없음.




In [ ]:
import seaborn as sns

plt.figure(figsize=(10,10)) #그래프 크기 확대
sns.heatmap(train.corr(), annot=True) #annot=True를 사용해 숫자 표기 가능

#hour와 한시간전 온도, 풍속, 미세먼지농도가 관련이 깊은것을 알수있음

# ***3. Data Cleansing & Pre-Processing***

In [ ]:
train.isna().sum() #결측치 확인
#결측치를 자동으로 처리해주는 모델도 존재하기는 함

In [ ]:
train[train['hour_bef_temperature'].isna()] #1시간전 기온에대한 결측 시간대가 자정과 오후 6시인것을 알수있음

#즉, 결측치를 평균으로 처리하기에는 두 온도가 차이가 클것이므로 무리가있음

In [ ]:
train.groupby('hour').mean()['hour_bef_temperature']

In [ ]:
train['hour_bef_temperature'].fillna({934:14.788136,1035:20.926667},inplace=True )

In [ ]:
plt.plot(train.groupby('hour').mean()['hour_bef_windspeed']) #시간에 따라 풍속의 차이가 큼을 알수있음

In [ ]:
train[train['hour_bef_windspeed'].isna()] #train에서 isna가 True인 것만 표시

In [ ]:
train.groupby('hour').mean()['hour_bef_windspeed']

In [ ]:
train[train['hour_bef_windspeed'].isna()].index

In [ ]:
train['hour_bef_windspeed'].fillna({18:3.281356 , 244:1.836667, 260:1.620000, 376:1.965517, 780:3.278333, 934:1.965517, 1035:3.838333, 1138:2.766667, 1229:1.633333},inplace=True)

In [ ]:
train[train['hour_bef_windspeed'].isna()] #전부 잘 채워진것을 확인

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
test[test['hour_bef_temperature'].isna()] 

In [ ]:
test['hour_bef_temperature'].fillna({653:19.704918},inplace=True) #train으로 냇던 평균을 그대로 사용

In [ ]:
test[test['hour_bef_windspeed'].isna()] 

In [ ]:
test['hour_bef_windspeed'].fillna({653:3.595082},inplace=True)

In [ ]:
test.isna().sum()

# ***4.Feature Engineering & Initial Modeling***

In [ ]:
features=['hour', 'hour_bef_temperature', 'hour_bef_windspeed']
X_train= train[features]
X_test=  test[features]
y_train= train['count']

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

# ***5.Model Tuning & Evaluation***

In [ ]:
model = RandomForestRegressor() #ctrl+spacebar 를 누르면 하이퍼 파라미터들을 확인가능

DecisionTreeRegressor()


RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)

RandomForestRegressor(n_estimators=100, criterion='mse', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=True, oob_score=False, n_jobs=None, random_state=None, verbose=0, warm_start=False, ccp_alpha=0.0, max_samples=None)

*   n_estimators: 질문을 묻고자 하는 나무의 개수(default:100)
*   n_jobs: 사용하고자 하는 CPU의 개수, 만약 -1로 설정하면 가지고있는 모든 CPU활용
*   max_depth: 얼마나 나무의 깊이를 길게 할것인지(모델의 과대적합을 방지하기 위해 사용)









In [ ]:
model_100= RandomForestRegressor(n_estimators=100,random_state=0)
model_100_5= RandomForestRegressor(n_estimators=100,max_depth=5, random_state=0)
model_200= RandomForestRegressor(n_estimators=200,random_state=0) 


이렇게 다양한 하이퍼 파라미터(옵션들)들을 사용해 분석하는 데이터에 가장 적합한 모델을 찾는과정을 Tuning이라 함

In [ ]:
model_100.fit(X_train,y_train)
model_100_5.fit(X_train,y_train)
model_200.fit(X_train,y_train)

In [ ]:
ypred1= model_100.predict(X_test)
ypred2= model_100_5.predict(X_test)
ypred3= model_200.predict(X_test)

In [ ]:
submission['count'] = ypred1
submission

In [ ]:
submission.to_csv('model_100.csv',index=False)

In [ ]:
submission['count'] = ypred2
submission.to_csv('model_100_5.csv',index=False)

In [ ]:
submission['count'] = ypred3
submission.to_csv('model_200.csv',index=False)